In [35]:
import casadi as ca
import numpy as np

In [36]:
T = 10
dt = 0.1
# N = int(T/dt)
N = 20

g = 9.81
x_dim = 10
u_dim = 4

w_max_yaw = 6.0
w_max_xy = 6.0
thrust_min = 2.0
thrust_max = 20.0

x_cost = np.diag([
    100, 100, 100,  
    10, 10, 10, 10,
    10, 10, 10]) 

u_cost = np.diag([0.1, 0.1, 0.1, 0.1])

In [ ]:
def runge_kutta_4(f, dt, x_dim, u_dim):
    M = 4
    DT = dt/M
    X0 = ca.SX.sym("X0", x_dim)
    U = ca.SX.sym("U", u_dim)

    X = X0
    Q = 0
    # --------- RK4------------
    for _ in range(M):
        k1, k1_q = f(X, U)
        k2, k2_q = f(X + DT/2 * k1, U)
        k3, k3_q = f(X + DT/2 * k2, U)
        k4, k4_q = f(X + DT * k3, U)
        X=X+DT/6*(k1 +2*k2 +2*k3 +k4)
        Q = Q + DT/6*(k1_q + 2*k2_q + 2*k3_q + k4_q)

    F = ca.Function('F', [X0, U], [X, Q],['x0','p'],['xf','qf'])
    return F


In [ ]:
px, py, pz = ca.SX.sym('px'), ca.SX.sym('py'), ca.SX.sym('pz')
qw, qx, qy, qz = ca.SX.sym('qw'), ca.SX.sym('qx'), ca.SX.sym('qy'), ca.SX.sym('qz')
vx, vy, vz = ca.SX.sym('vx'), ca.SX.sym('vy'), ca.SX.sym('vz')

thrust, wx, wy, wz = ca.SX.sym('thrust'), ca.SX.sym('wx'), ca.SX.sym('wy'), ca.SX.sym('wz')

x = ca.vertcat(px, py, pz, qw, qx, qy, qz, vx, vy, vz) 
u = ca.vertcat(thrust, wx, wy, wz)

x_dot = ca.vertcat(
    vx,
    vy,
    vz,
    0.5 * ( -wx*qx - wy*qy - wz*qz ),
    0.5 * (  wx*qw + wz*qy - wy*qz ),
    0.5 * (  wy*qw - wz*qx + wx*qz ),
    0.5 * (  wz*qw + wy*qx - wx*qy ),
    2 * ( qw*qy + qx*qz ) * thrust,
    2 * ( qy*qz - qw*qx ) * thrust, 
    (qw*qw - qx*qx -qy*qy + qz*qz) * thrust - g
    # (1 - 2*qx*qx - 2*qy*qy) * thrust - self._gz
)

f = ca.Function('f', [x, u], [x_dot], ['x', 'u'], ['ode'])

F = runge_kutta_4(f, dt, x_dim, u_dim)
fMap = F.map(N, "openmp") # parallel

Fk = F(x0=[0.2,0.3],p=0.4)
print(Fk['xf'])
print(Fk['qf'])

CasADi - 2024-12-06 14:26:09 WARNING("CasADi was not compiled with WITH_OPENMP=ON. Falling back to serial evaluation.") [.../casadi/core/map.cpp:403]


RuntimeError: Error in Function::call for 'F' [SXFunction] at .../casadi/core/function.cpp:1432:
.../casadi/core/function_internal.hpp:1091: FunctionInternal::index_in: could not find entry "p". Available names are: [i0, i1].

In [25]:
delta_x = ca.SX.sym("delta_x", x_dim)
delta_u = ca.SX.sym("delta_u", u_dim)        

cost_x = delta_x.T @ x_cost @ delta_x 
cost_u = delta_u.T @ u_cost @ delta_u

f_cost_x = ca.Function('cost_x', [delta_x], [cost_x])
f_cost_u = ca.Function('cost_u', [delta_u], [cost_u])

In [26]:
nlp_w = []       # nlp variables
nlp_w0 = []      # initial guess of nlp variables
lbw = []         # lower bound of the variables, lbw <= nlp_x
ubw = []         # upper bound of the variables, nlp_x <= ubw

mpc_obj = 0      # objective 
nlp_g = []       # constraint functions
lbg = []         # lower bound of constrait functions, lbg < g
ubg = []         # upper bound of constrait functions, g < ubg

u_min = [thrust_min, -w_max_xy, -w_max_xy, -w_max_yaw]
u_max = [thrust_max,  w_max_xy,  w_max_xy,  w_max_yaw]

x_min = [-ca.inf for _ in range(x_dim)]
x_max = [+ca.inf for _ in range(x_dim)]

g_min = [0 for _ in range(x_dim)]
g_max = [0 for _ in range(x_dim)]

In [34]:
print(x_dim+(x_dim+3)*N+x_dim)

P = ca.SX.sym("P", x_dim+(x_dim+3)*N+x_dim)
X = ca.SX.sym("X", x_dim, N+1)
U = ca.SX.sym("U", u_dim, N)
#
print(P.shape)

# nlp_w += [X[:, 0]]
# nlp_w0 += self._quad_s0
# lbw += x_min
# ubw += x_max

# # # starting point.
# nlp_g += [ X[:, 0] - P[0:x_dim]]
# lbg += g_min
# ubg += g_max

670
(670, 1)
